<a href="https://colab.research.google.com/github/debojit11070/ai-engineering-aws/blob/main/embaddings_for_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from importlib.metadata import version
import tiktoken
import torch

In [2]:
print(version("torch"))

2.6.0+cu124


In [3]:
print(version("tiktoken"))

0.9.0


##Loading the dataset

In [4]:
# Reading the whole file
with open("/content/the-verdict.txt", "r", encoding="latin-1") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [5]:
# tokenizing the raw text
import re
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [6]:
len(preprocessed)

4649

##convert tokens into token ids
